# Table of Contents
 <p><div class="lev2 toc-item"><a href="#group-by-function;-reducer-generator" data-toc-modified-id="group-by-function;-reducer-generator-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>group by function; reducer generator</a></div>

In [167]:
sql="""
SELECT cc, sum(iso_num) AS x, sum(cc) AS xm, sum(avg_vis) AS x,'ddd' as d, avg_vis
FROM 'NOAA/DMSP-OLS/NIGHTTIME_LIGHTS'
WHERE ST_INTERSECTS(ST_SetSRID(ST_GeomFromGeoJSON('{\"type\":\"Polygon\",\"coordinates\":[[[-5.273512601852417,42.81137220349083],[-5.273512601852417,42.811803118457306],[-5.272732079029083,42.811803118457306],[-5.272732079029083,42.81137220349083],[-5.273512601852417,42.81137220349083]]]}'), 4326), the_geom) 
and iso_num > 2 
and iso_num < 10 
or iso_num = 2
order by y asc
GROUP BY x 
LIMIT 1
"""
url='https://api.resourcewatch.org/v1/convert/sql2SQL'
payload={
    'sql':sql
}
r=requests.get(url, params=payload).json()
r

{'data': {'attributes': {'jsonSql': {'from': "'NOAA/DMSP-OLS/NIGHTTIME_LIGHTS'",
    'group': [{'type': 'literal', 'value': 'x'}],
    'limit': 1,
    'orderBy': [{'alias': None,
      'direction': 'asc',
      'type': 'literal',
      'value': 'y'}],
    'select': [{'alias': None, 'type': 'literal', 'value': 'cc'},
     {'alias': 'x',
      'arguments': [{'type': 'literal', 'value': 'iso_num'}],
      'type': 'function',
      'value': 'sum'},
     {'alias': 'xm',
      'arguments': [{'type': 'literal', 'value': 'cc'}],
      'type': 'function',
      'value': 'sum'},
     {'alias': 'x',
      'arguments': [{'type': 'literal', 'value': 'avg_vis'}],
      'type': 'function',
      'value': 'sum'},
     {'alias': 'd', 'type': 'string', 'value': "'ddd'"},
     {'alias': None, 'type': 'literal', 'value': 'avg_vis'}],
    'where': {'left': {'left': {'left': {'alias': None,
        'arguments': [{'alias': None,
          'arguments': [{'alias': None,
            'arguments': [{'type': 'stri

In [144]:
import ee
import json
from cached_property import cached_property
ee.Initialize()

In [160]:
#le pasariamos data.attributes.jsonSql.where nos deberia de devolver null si no lo encuentra o el geojson. type:function, value:ST_GeomFromGeoJSON arguments[0] value <-- json.parse
def filterGen(data, parents=[]):
#     print(data)
#     print('-')
#     print(parents)
#     print('****')
    _filters = {
            '<': ee.Filter.lt,
            '<=': ee.Filter.lte,
            '>': ee.Filter.gt,
            '>=': ee.Filter.gte,
            '<>': ee.Filter.neq,
            '=': ee.Filter.eq,
            '!=': ee.Filter.neq,
            'bedate': ee.Filter.date,
            'between': ee.Filter.rangeContains,
            'like': ee.Filter.eq,
            '%like%': ee.Filter.stringContains,
            '%like': ee.Filter.stringEndsWith,
            'like%': ee.Filter.stringStartsWith
            
        }
    _comparisons = {
            'and': ee.Filter.And,
            'or': ee.Filter.Or
        }
    left = None
    right = None
    result =[]
    dparents=list(parents)
    
    if 'type' in [*data] and data['type']=='conditional': ########---------------------------- Leaf with groups:
        left = data['left']
        right = data['right']
        dparents=[data['value']]
    
    elif 'type' in [*data] and data['type']=='operator': ########------------------------------------------- Latests leaf we will want to return  
        return data['value']
    
    if left and right: ########-------------------------------------- leaf group iteration
        #for l in left:
        partialL=filterGen(left, dparents)
        #for r in right:
        partialR=filterGen(right, dparents)
        
        if not partialL:
            result=partialR
        elif not partialR:
            result=partialL
        else:
            result={dparents[0]:[partialR, partialL]}
        
    return result ########----------------------------------- Return result in:[[ee.Filter.eq('month','historical'),ee.Filter.eq('model','historical'),...],...]             

geometry=None        
x=filterGen(r['data']['attributes']['jsonSql']['where'])
print(x)

{'or': ['=', {'and': ['<', '>']}]}


In [172]:
def notDupe(list):
    r = [i for n, i in enumerate(list) if i not in list[n + 1:]]
    return len(list) == len(r)

def _colGen(selectArray):
    columns={'columns': {'cc': 'String',
              'iso_alpha2': 'String',
              'iso_alpha3': 'String',
              'iso_num': 'Float',
              'name': 'String',
              'region': 'String',
              'system:index': 'String',
              'tld': 'String'}}
    
    bands={'bands':[{'id': 'avg_vis', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}, 
                    {'id': 'stable_lights', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}, 
                    {'id': 'cf_cvg', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}, 
                    {'id': 'avg_lights_x_pct', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}]}
    c=[]
    b=[]
    if columns:
        _init_cols=columns['columns'].keys()
    if bands:
        _init_bands=[v['id'] for v in bands['bands']]
    
    response={
        'columns':[],
        '_columns':[],
        'bands':[],
        '_bands':[],
        'functions':[],
        'others':[]
    }
    for a in selectArray:
        if a['type']=='literal':
            # This will retrieve the columns and bands for our dataset
            if a['value'] in _init_cols:
                response['columns'].append(a)
            elif a['value'] in _init_bands:
                response['bands'].append(a)
            else:
                raise NameError('column/band name not valid: {0}'.format(a['value']))
            
        elif a['type']=='function':
            # This will retrieve the columns and bands for our dataset and extend the cols/bands to select if they already hasn't being selected
            response['functions'].append(a)
            c.extend([args['value'] for args in a['arguments'] if args['type']=='literal' and args['value'] in _init_cols])
            b.extend([args['value'] for args in a['arguments'] if args['type']=='literal'and args['value'] in _init_bands])
            f=[args['value'] for args in a['arguments'] if args['type']=='literal'and args['value'] not in _init_bands and args['value'] not in _init_cols]
            if f:
                raise NameError('column/band name not valid in function {0}: {1}'.format(a['value'],f))
        elif a['type']=='wildcard':
            d = [{'type':'literal','alias': None, 'value':f} for f in _init_cols]
            response['columns'].extend(d)
        else:
            response['others'].append(a)
    
    assert notDupe(response['columns']), 'we cannot have 2 columns with the same alias'
    assert notDupe(response['bands']), 'we cannot have 2 bands with the same alias'
    
    response['_columns'] =list(set([a['value'] for a in response['columns']]).union(c))
    response['_bands']=list(set([a['value'] for a in response['bands']]).union(b))
    
    return response
x=_colGen(r['data']['attributes']['jsonSql']['select'])
print(x)

{'columns': [{'value': 'cc', 'alias': None, 'type': 'literal'}], '_columns': ['cc', 'iso_num'], 'bands': [{'value': 'avg_vis', 'alias': None, 'type': 'literal'}], '_bands': ['avg_vis'], 'functions': [{'type': 'function', 'alias': 'x', 'value': 'sum', 'arguments': [{'value': 'iso_num', 'type': 'literal'}]}, {'type': 'function', 'alias': 'xm', 'value': 'sum', 'arguments': [{'value': 'cc', 'type': 'literal'}]}, {'type': 'function', 'alias': 'x', 'value': 'sum', 'arguments': [{'value': 'avg_vis', 'type': 'literal'}]}], 'others': [{'value': "'ddd'", 'alias': 'd', 'type': 'string'}]}


For management pourposes we will need to get the list of bands; columns functions and others selected on the query, compare it with the available columns/bands and decide what to do next.

For feature collection we will only have columns: so function will be applied to column values and we will need to reduceColumn.  
For image collections we will have columns and bands we migh need to reduce by column or reduceImage with reduceRegion.  
For images we will only have bands. reducers (functions inside ReduceRegion) will be applied only to bands. 



## group by function; reducer generator

In [223]:
#Load a collection of US counties with census data properties.
counties = ee.FeatureCollection('ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM');

#Compute sums of the specified properties, grouped by state name.
sums = counties.filter(ee.Filter.And(ee.Filter.neq('Census 2000 Population', None), ee.Filter.neq('Census 2000 Housing Units', None))).reduceColumns(**{
    'selectors': ['Census 2000 Population', 'Census 2000 Housing Units', 'StateName'],
    'reducer': ee.Reducer.sum().repeat(2).group(**{
      'groupField': 2,
      'groupName': 'state'
    })
});

#Print the resultant Dictionary.
print(sums);

ee.Dictionary({
  "type": "Invocation",
  "arguments": {
    "collection": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "tableId": "ft:1S4EB6319wWW2sWQDPhDvmSBIVrD3iEmCLYB7nMM"
          },
          "functionName": "Collection.loadTable"
        },
        "filter": {
          "type": "Invocation",
          "arguments": {
            "filters": [
              {
                "type": "Invocation",
                "arguments": {
                  "filter": {
                    "type": "Invocation",
                    "arguments": {
                      "leftField": "Census 2000 Population"
                    },
                    "functionName": "Filter.equals"
                  }
                },
                "functionName": "Filter.not"
              },
              {
                "type": "Invocation",
                "arguments": {
                  "filter": {
      

input: 
* `x`
* `r['data']['attributes']['jsonSql']['group']`

output: 
```
ee.Reducer.sum().repeat(3).group(**{
            'groupField': 2,
            'groupName': 'x',
    }),

```

In [229]:
print('x: ',x['functions'])

print('group by: ',r['data']['attributes']['jsonSql']['group'])

x:  [{'type': 'function', 'alias': 'x', 'value': 'sum', 'arguments': [{'value': 'iso_num', 'type': 'literal'}]}, {'type': 'function', 'alias': 'xm', 'value': 'sum', 'arguments': [{'value': 'cc', 'type': 'literal'}]}, {'type': 'function', 'alias': 'x', 'value': 'sum', 'arguments': [{'value': 'avg_vis', 'type': 'literal'}]}]
group by:  [{'type': 'literal', 'value': 'x'}, {'type': 'literal', 'value': 'y'}]


In [263]:
def reduceGen(selectFunctions, groupBy):
    #input: group by and above x result
    #output: reducer well compose
    _agFunctions = {
            'mean': ee.Reducer.mean,
            'max': ee.Reducer.max,
            'min': ee.Reducer.min,
            'count': ee.Reducer.count,
            'sum': ee.Reducer.sum
        }
    reducers={'reduceRegion':{},
              'reduceColumns':{},
              'reduceImage':{}
    }
    
    reducers = []
    result = []
    
    if any(function['value'] == 'sum' for function in selectFunctions):
        sum_quantity = sum(function['value'] == 'sum' for function in selectFunctions)
        reducer = _agFunctions['sum']().repeat(sum_quantity)
        reducers.append(reducer)
        
    # For each reducer in result, apply each group by
    # .group(**{'groupField': 0,'groupName':groupBy[0]['value']}
    
    for reducer in reducers:
        for group in groupBy:
            reducer = reducer.group(**{'groupField': 1,'groupName':group['value']})
            if group == groupBy[-1]:
                result.append(reducer)
        
    return result
    
    
result = reduceGen(x['functions'], r['data']['attributes']['jsonSql']['group'])
print(result[0].getInfo())

{'type': 'Reducer.group', 'reducer': {'type': 'Reducer.group', 'reducer': {'type': 'Reducer.repeat', 'reducer': {'type': 'Reducer.sum'}, 'count': 3}, 'groupField': 1, 'groupName': 'x'}, 'groupField': 1, 'groupName': 'y'}


In [122]:
class SQL2GEE(object):
    """
    Takes an SQL2JSON like object and relates it to Google's Earth Engine syntax (specifically the Python 3.5 GEE API).
    Designed to perform operations on three types of geo-objects, Tables (Feature Collections), Rasters (Images) or 
    Raster Collections (Image Collections).
    
    For single rasters there are only a specific number of valid operations (retrieve metadata, histogram data, or get
    summary statistics). We use postgis-like functions as the syntax to do this, and check to see if this is given in
    the sql string to detect the user intention.
    
    If geojson data is provided, we will assume that a user intends for us to subset an image, by these data.
    """
    def __init__(self, sql, geojson=None, flags=None):
        self._raw = sql
        self._asset_id = sql['data']['attributes']['jsonSql']['from'].strip("'")
        self.metadata = self._metadata()
        self.type = self.metadata['type']
        self._asset =self._getAsset()
        self._parsed = sql['data']['attributes']['jsonSql']
        self.geojson = self._geojson_to_featurecollection(geojson)
        self.flags = flags  # <-- Will be used in a later version of the code
        self._filters = {
            '<': ee.Filter.lt,
            '<=': ee.Filter.lte,
            '>': ee.Filter.gt,
            '>=': ee.Filter.gte,
            '<>': ee.Filter.neq,
            '=': ee.Filter.eq,
            '!=': ee.Filter.neq,
            'bedate': ee.Filter.date,
            'between': ee.Filter.rangeContains,
            'like': ee.Filter.eq,
            '%like%': ee.Filter.stringContains,
            '%like': ee.Filter.stringEndsWith,
            'like%': ee.Filter.stringStartsWith
            
        }
        self._comparisons = {
            'and': ee.Filter.And,
            'or': ee.Filter.Or
        }
        self._agFunctions = {
            'mean': ee.Reducer.mean,
            'max': ee.Reducer.max,
            'min': ee.Reducer.min,
            'count': ee.Reducer.count,
        }
        self._agTFunctions = {
            'mean': ee.Reducer.mean,
            'max': ee.Reducer.max,
            'min': ee.Reducer.min,
            'min': ee.Reducer.min,
        }
        
    def _metadata(self):
        """Property that holds the Metadata dictionary returned from Earth Engine."""
        if 'ft:' in self._asset_id:
            meta = ee.FeatureCollection(self._asset_id).limit(0).getInfo()
            assert meta != None, 'please enter a valid fusion table'
            info={'type': meta['type'],
                 'columns':meta['columns'],
                 'id':self._asset_id,
                 'version':'',
                 'properties':meta['properties']}
            return info
        else:
            info=ee.data.getInfo(self._asset_id)
            assert info!=None, "data type not expected"
            if ('bands' in info) and (not info['bands']):
                meta=ee.ImageCollection(self._asset_id).limit(1).getInfo()['features'][0] ### this is a bit ... 
                info['bands']=meta['bands']
                info['columns']={k: type(v).__name__  for k,v in meta['properties'].items()}
            
            return info  
    
    def _geo_extraction(self, json_input):
        lookup_key='type'
        lookup_value='function'
        Sqlfunction='ST_GeomFromGeoJSON'
        if isinstance(json_input, dict):
            for k, v in json_input.items():
                if k == lookup_key and v==lookup_value and json_input['value']==Sqlfunction:
                    yield json_input['arguments'][0]['value'].strip("'")
                else:
                    for child_val in self._geo_extraction(v):
                        yield child_val
        elif isinstance(json_input, list):
            for item in json_input:
                for item_val in self._geo_extraction(item):
                    yield item_val
    
    def _geojson_to_featurecollection(self, geojson):
        """If Geojson kwarg is recieved or ST_GEOMFROMGEOJSON sql argument is used,
        (containing geojson data) convert it into a useable E.E. object."""
        geometries=[json.loads(x) for x in self._geo_extraction(self._parsed)]
        if geometries:
            geojson = {u'features': geometries,
                   u'type': u'FeatureCollection'}
        if isinstance(geojson, dict):
            assert geojson.get('features') != None, "Expected key not found in item passed to geojoson"
            return ee.FeatureCollection(geojson.get('features'))
        else:
            return None
    def _getAsset(self):
        _data={
            'Image':ee.Image(self._asset_id),
            'FeatureCollection':ee.FeatureCollection(self._asset_id),
            'ImageCollection':ee.ImageCollection(self._asset_id)
        }
        return _data[self.type]
    
    def _filterGen(self, data, parents=[]):
        left = None
        right = None
        result =[]
        dparents=list(parents)
        if 'type' in [*data] and data['type']=='conditional': ########---------------------------- Leaf with groups:
            left = data['left']
            right = data['right']
            dparents=[data['value']]
        elif 'type' in [*data] and data['type']=='operator': ########------------------------------------------- Latests leaf we will want to return  
            return self._filters[data['value']](data['left']['value'], data['right']['value'])
            #return {data['value']:[data['left']['value'], data['right']['value']]}

        if left and right: ########-------------------------------------- leaf group iteration
            #for l in left:
            partialL=self._filterGen(left, dparents)
            #for r in right:
            partialR=self._filterGen(right, dparents)
            if not partialL:
                result=partialR
            elif not partialR:
                result=partialL
            else:
                result=self._comparisons[dparents[0]](partialR, partialL)
            #result={dparents[0] : [*partialR, *partialL]}
        
        return result ########----------------------------------- Return result in:[[ee.Filter.eq('month','historical'),ee.Filter.eq('model','historical'),...],...]
    
    def _select(self,cols):
        # It returns the asset with only the *selected* columns/bands.
        
        if self['type']:
            return self.asset.select(cols)
        else:
            return self.asset
    
    def _where(self):
        # It gets *where* conditions and converts them in the proper filters.
        # self.asset.filter(ee.Filter([ee.Filter.eq('scenario','historical'), ee.Filter.date('1996-01-01','1997-01-01')])).filterBounds(geometry)
        _filters=self._filterGen(self._parsed['where'])
        if self.geojson:
            return self._asset.filter(_filters).filterBounds(self.geojson)
        else:
            return self._asset.filter(_filters)
        
        return 0
    
    def _fGroupBy(self):
        return 0
    
    def _orderBy(self):
        return 0
    
    def _limit(self):
        return 0 
    
    def _genSql(self):
        # it will depends on the asset type, but it will give back this:
        # imageCollection.<filters>.<functions>.<sorts>.<imageReducers>.<reduceRegion>.limit(n).getInfo()
        # featureCollection.<filters>.<functions>.<sorts>.<columnReducers>.limit(n).getInfo()
        # image.<filters>.<functions>.<sorts>.<reduceRegion>.limit(n).getInfo()
        return 0
        

In [120]:
import requests

In [123]:
f=SQL2GEE(r)
print(f.metadata)

{'type': 'ImageCollection', 'bands': [{'id': 'avg_vis', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}, {'id': 'stable_lights', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}, {'id': 'cf_cvg', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}, {'id': 'avg_lights_x_pct', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [43201, 16801], 'crs': 'EPSG:4326', 'crs_transform': [0.0083333333, 0.0, -180.00416666665, 0.0, -0.0083333333, 75.00416666665]}], 'id': 'NOAA

In [36]:
#print(f.geojson)
#
print(f._filterGen(r['data']['attributes']['jsonSql']['where']))
print(f._filterGen(r['data']['attributes']['jsonSql']['where']).getInfo())

ee.Filter({
  "type": "Invocation",
  "arguments": {
    "filters": [
      {
        "type": "Invocation",
        "arguments": {
          "leftField": "iso_num",
          "rightValue": 2
        },
        "functionName": "Filter.equals"
      },
      {
        "type": "Invocation",
        "arguments": {
          "filters": [
            {
              "type": "Invocation",
              "arguments": {
                "leftField": "iso_num",
                "rightValue": 10
              },
              "functionName": "Filter.lessThan"
            },
            {
              "type": "Invocation",
              "arguments": {
                "leftField": "iso_num",
                "rightValue": 2
              },
              "functionName": "Filter.greaterThan"
            }
          ]
        },
        "functionName": "Filter.and"
      }
    ]
  },
  "functionName": "Filter.or"
})
{'type': 'Filter.or', 'filters': [{'type': 'Filter.eq', 'rightValue': 2, 'leftField': 'i

In [37]:
f._where().getInfo()

{'columns': {'cc': 'String',
  'iso_alpha2': 'String',
  'iso_alpha3': 'String',
  'iso_num': 'Float',
  'name': 'String',
  'region': 'String',
  'system:index': 'String',
  'tld': 'String'},
 'features': [{'geometry': {'coordinates': [[[19.368690486799363,
       41.89036111360716],
      [19.369028499485896, 41.88943152137818],
      [19.369873804523127, 41.88892446877057],
      [19.371732881687077, 41.888501900345474],
      [19.377056913995247, 41.88850192625604],
      [19.379845754785645, 41.88765684239492],
      [19.381367120629598, 41.88681174211565],
      [19.381958658277213, 41.8861356357676],
      [19.38208391875729, 41.885028826262904],
      [19.380691078475213, 41.88250167432816],
      [19.376380787006532, 41.877008449547205],
      [19.375366643654427, 41.8752337444261],
      [19.37527987018657, 41.87152141057877],
      [19.375673800069887, 41.86805659448535],
      [19.376042769245288, 41.865768619581175],
      [19.375958537143404, 41.86255721879424],
      [19